In [1]:
import numpy as np
import pickle
from delta_classes import Cell, SubCell, Chamber

# open the results from Delta pipeline
with open("data0.pickle", "rb") as file:
    data_object = pickle.load(file)


In [2]:

# prepare dictionary to create cells

extract_data_list = []
for lineage_list_no in range(len(data_object['res'])):
    extract_data_list.append([])
    for cell_number, cell_res in enumerate((data_object['res'][lineage_list_no]['lineage'])):
        lineage_dict = dict(
            cell_number = cell_number+1,
            mother_nb = cell_res['mothernb'],
            chamber_nb = lineage_list_no+1,
            framenbs = cell_res['framenbs'],
            daughters = cell_res['daughters'],
            length = cell_res['length'],
            width = cell_res['width'],
            area = cell_res['area'],
            fluorescence = cell_res['fluo1']
            )
        extract_data_list[lineage_list_no].append(lineage_dict)



In [3]:
# create chamber family tree
all_family_trees = []
for chamber in extract_data_list:
    family_tree = []
    for cell in chamber:
        lineage_db = dict(
            cell_nb = cell['cell_number'],
            mother_nb = cell['mother_nb']
        )
        family_tree.append(lineage_db)
    all_family_trees.append(family_tree)




In [4]:
# initialize all cells and store them in experiment
#experiment = []
chambers = []
total_cell = 0
for count, chamber in enumerate(extract_data_list):
    current_chamber_number = count+1
    cells_in_chamber = []
    for cell in chamber:
        current_cell = Cell(
            total_cell,
            cell['cell_number'],
            cell['mother_nb'], 
            cell['chamber_nb'],
            cell['framenbs'],
            cell['daughters'],
            cell['length'],
            cell['width'],
            cell['area'],
            cell['fluorescence'],
            all_family_trees[count])
        #experiment.append(current_cell)
        total_cell += 1
        cells_in_chamber.append(current_cell)
    chamber_object = Chamber(current_chamber_number, cells_in_chamber)
    chambers.append(chamber_object)




In [5]:
# Map out lineage trees here
for chamber in chambers:
    for cell in chamber.cells:
        cell.make_ancestry_map()
        cell.find_lineage()



In [6]:
# calculate fluorescence by area
for chamber in chambers:
    for cell in chamber.cells:
        cell.calculate_fluorescence_by_area()



In [7]:
# Calculate change in variables over time
for chamber in chambers:
    for cell in chamber.cells:
        cell.calculate_area_growth()
        cell.calculate_length_growth()
        cell.calculate_width_growth()
        cell.calculate_fluorescence_growth()
        cell.calculate_fluorescence_growth_by_area()



In [8]:
# Create subcell object and append it to cells        
for chamber in chambers:
    for cell in chamber.cells:
        cell.create_subCells()



In [10]:
# find average growth rate of all subcells
average_growth_length = []
for chamber in chambers:
    for cell in chamber.cells:
        for subcell in cell.subcells:
            current_average_growth_length = np.mean(subcell.length_growth)
            if not np.isnan(current_average_growth_length): #nans produced by subcells.length_growth = []
                average_growth_length.append(current_average_growth_length)

average_growth_length_per_minute = np.mean(average_growth_length)
            
average_growth_length_per_minute

0.4626953801283677

In [17]:
# sometimes the software fails to detect splitting events? or maybe it is the cell exiting the chamber?
chambers[6].cells[33].daughters
chambers[6].cells[33].length

# how often does this happen if we assume that the wrong identification causes slope of -1
len(average_growth_length)
event_counter = 0
for chamber in chambers:
    for cell in chamber.cells:
        for subcell in cell.subcells:
            for value in subcell.length_growth:
                if value < -1:
                    event_counter += 1
print(event_counter)
# maybe 152 out of 2369 subcells or 6.4% of cells. Again that is assuming that cells
# Only decrease because of 

# Find out why nans are made
average_growth_length = []
for chamber in chambers:
    for cell in chamber.cells:
        for subcell in cell.subcells:
            current_average_growth_length = np.mean(subcell.length_growth)
            if np.isnan(current_average_growth_length):
                cell.chamber_nb, cell.cell_number, subcell.subcell_nb

# Nans are made when cells split right before it exits resulting in empty length_growth array
# examples of nan producers
print(f"examples of nan producers {(chambers[0].cells[0].subcells[26].length_growth, chambers[15].cells[59].subcells[1].length_growth, chambers[15].cells[59].daughters)})")

152
examples of nan producers (array([], dtype=float64), array([], dtype=float64), array([ 0.,  0.,  0.,  0.,  0., 62.])))
